# Importation

In [119]:
from tqdm import tqdm
import time
import warnings
warnings.filterwarnings("ignore")
import sys
import re
import pandas as pd
from PyPDF2 import PdfReader
import cv2
import numpy as np
from PIL import ImageGrab
import time
import os

In [3]:
from pdf2image import convert_from_path

In [69]:
pip install pyautogui

     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.2 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.2 kB ? eta -:--:--
     ------------ ------------------------- 20.5/61.2 kB 131.3 kB/s eta 0:00:01
     ------------------- ------------------ 30.7/61.2 kB 130.4 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/61.2 kB 178.6 kB/s eta 0:00:01
     -------------------------------------- 61.2/61.2 kB 217.0 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to buil

In [70]:
import pyautogui

# Transforming to pictures

In [12]:
# PDF file paths
pdfPMBOK = "PMBOK6-2017.pdf" #431-494
pdfPMI = "practice-standard-project-risk-management.pdf" #1-116
output_folder_1="PMBOOKimgs"
output_folder_2="Practiceimgs"

In [42]:
def convert_pdf_to_images(pdf_file_path, output_folder, start_page, end_page):
    images = convert_from_path(pdf_file_path, output_folder=output_folder, fmt='png', first_page=start_page, last_page=end_page, output_file="page")

    print("Done.")

In [43]:
convert_pdf_to_images(pdfPMBOK, output_folder_1, 431, 494)

Done.


In [44]:
convert_pdf_to_images(pdfPMI, output_folder_2, 1, 116)

Done.


# Extracting Figures

### YOLO: (does not work)

###### must install yolov3.weights yolov3.cfg coco.names (ask chatgpt)

In [80]:
def yolo_object_detection(image_path):
    # Load YOLO
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    classes = []
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    layer_names = net.getUnconnectedOutLayersNames()

    # Load image
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    # Preprocess image for YOLO
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(layer_names)

    # Get class IDs, confidences, and bounding boxes
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            print(f"Confidence: {confidence}")
            if confidence > 0.1 and class_id == 0:  # Class ID 0 is "person"
                center_x, center_y, w, h = (detection[:4] * np.array([width, height, width, height])).astype(int)
                x, y = int(center_x - w / 2), int(center_y - h / 2)
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, int(w), int(h)])

    # Apply non-maximum suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Save screenshots for detected regions
    print(indices)

    for i in indices:
        i = i[0]
        box = boxes[i]
        x, y, w, h = box

        # Take a screenshot of the detected region using pyautogui
        screenshot = pyautogui.screenshot(region=(x, y, w, h))
        screenshot.save(f"screenshot_{i}.png")

In [81]:
yolo_object_detection("PMBOOKimgs/page0001-432.png")

Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confiden

### openCV:

In [122]:
def detect_shapes(image_path, min_contour_area=1000):
    # Read the image
    image = cv2.imread("PMBOOKimgs/"+image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply GaussianBlur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Use Canny edge detection
    edges = cv2.Canny(blurred, 50, 150)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    k=0
    for contour in contours:
        k=k+1
        # Calculate the contour area
        area = cv2.contourArea(contour)

        # If the contour area is greater than the minimum threshold
        if area >= min_contour_area:
            # Approximate the contour as a polygon
            epsilon = 0.04 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)

            # Check if the polygon has 4 vertices, indicating a rectangle
            if len(approx) == 4:
                # Get the bounding box coordinates of the rectangle
                x, y, w, h = cv2.boundingRect(contour)

                # Define a region of interest (ROI) using the rectangle coordinates
                roi = image[y:y+h, x:x+w]

                # Save the ROI as an image for inspection
                cv2.imwrite("screenshots/"+image_path+"/"+str(k)+"detected__"+image_path, roi)                
                

In [123]:
detect_shapes("page0001-445.png")

In [124]:
for k in os.listdir("PMBOOKimgs/"):
    os.makedirs("screenshots/"+k, exist_ok=True)
    detect_shapes(k)

# Extracting Texts

In [ ]:
# Extracting Figures